In [ ]:
import akshare as ak
import pandas as pd
import sqlite3
from datetime import datetime
from tqdm import tqdm
import os
import time

# [StorageEngine 保持不变，无需修改]
class StorageEngine:
    def __init__(self, db_path='data/stock_market.db'):
        os.makedirs(os.path.dirname(db_path), exist_ok=True)
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()

    def save_to_sql(self, df, table_name, if_exists='replace'):
        try:
            df.to_sql(table_name, self.conn, if_exists=if_exists, index=False)
            print(f"[Storage] Table '{table_name}' saved. Rows: {len(df)}")
        except Exception as e:
            print(f"[Storage] Error saving {table_name}: {e}")

    def load_from_sql(self, table_name):
        try:
            return pd.read_sql(f"SELECT * FROM {table_name}", self.conn)
        except Exception as e:
            # print(f"[Storage] Table {table_name} not found.")
            return pd.DataFrame()
    
    def table_exists(self, table_name):
        self.cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
        return self.cursor.fetchone() is not None
    
    def get_latest_date(self, table_name, date_column='fetch_date'):
            """获取表中最新的记录日期"""
            if not self.table_exists(table_name):
                return None
            try:
                self.cursor.execute(f"SELECT MAX({date_column}) FROM {table_name}")
                return self.cursor.fetchone()[0]
            except:
                return None
    def close(self):
        self.conn.close()

class DataEngine:
    def __init__(self):
        self.request_interval = 0.5 

    def fetch_market_snapshot(self):
        # ... (这部分保持不变) ...
        print("[DAQ] Fetching Real-time Market Snapshot...")
        df_spot = ak.stock_zh_a_spot_em()
        rename_map = {
            '代码': 'symbol', '名称': 'name', 
            '最新价': 'price', '涨跌幅': 'change_pct',
            '市盈率-动态': 'pe_ttm', '市净率': 'pb', 
            '总市值': 'market_cap', '换手率': 'turnover'
        }
        df_spot = df_spot.rename(columns=rename_map)
        numeric_cols = ['pe_ttm', 'pb', 'market_cap', 'change_pct']
        for col in numeric_cols:
            if col in df_spot.columns:
                df_spot[col] = pd.to_numeric(df_spot[col], errors='coerce')
        return df_spot

    def fetch_detailed_financials(self, symbol_list):
        print(f"[DAQ] Starting Deep Reconstruction for {len(symbol_list)} stocks...")
        financial_data = []
        
        for symbol in tqdm(symbol_list):
            try:
                df = None
                source = 'em'
                
                # 1. 尝试 EM
                try:
                    df = ak.stock_financial_analysis_indicator(symbol=symbol)
                    if df is None or df.empty: raise ValueError
                except:
                    # 2. 尝试 THS
                    try:
                        source = 'ths'
                        df = ak.stock_financial_abstract_ths(symbol=symbol, indicator="按报告期")
                    except:
                        continue 

                if df is None or df.empty: continue

                # ==========================================================
                # 关键修正 1: 强制时间排序 (Time Ordering)
                # ==========================================================
                # 无论哪个接口，先找到日期列，转为 datetime 并降序排列
                date_col = None
                for col in df.columns:
                    if '日期' in str(col) or '报告期' in str(col):
                        date_col = col
                        break
                
                if date_col:
                    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
                    df = df.sort_values(by=date_col, ascending=False) # 降序：最新的在最上面
                
                # 取最新的一行 (Now this is truly the latest data)
                latest = df.iloc[0]

                # ==========================================================
                # 关键修正 2: 超强鲁棒的取值函数 (Handle Boolean)
                # ==========================================================
                def get_fuzzy(keywords_list):
                    for col_name in latest.index:
                        if any(k in str(col_name) for k in keywords_list):
                            raw_val = latest[col_name]
                            
                            # 处理布尔值 False/True
                            if isinstance(raw_val, bool):
                                return 0.0
                                
                            # 处理字符串清洗
                            val_str = str(raw_val).replace('%', '').replace(',', '').replace('万', '').replace('亿', '')
                            try:
                                return float(val_str)
                            except:
                                return 0.0
                    return 0.0

                data_point = {'symbol': symbol}
                data_point['roe'] = get_fuzzy(['净资产收益率', 'ROE'])
                data_point['gross_margin'] = get_fuzzy(['毛利率', '销售毛利'])
                data_point['net_profit_growth'] = get_fuzzy(['净利润同比增长', '净利润增长', '归属净利润同比'])
                data_point['debt_ratio'] = get_fuzzy(['资产负债率', '负债合计'])
                data_point['eps'] = get_fuzzy(['基本每股收益', '每股收益'])
                data_point['ocf_per_share'] = get_fuzzy(['每股经营活动', '每股经营现金', '经营净现金流/股'])
                
                eps = data_point['eps']
                data_point['cash_to_profit_ratio'] = (data_point['ocf_per_share'] / eps) if eps != 0 else 0
                
                financial_data.append(data_point)
                time.sleep(0.3 if source == 'em' else 0.1)
                
            except Exception as e:
                # print(f"[Error] {symbol}: {e}")
                continue
                    
        return pd.DataFrame(financial_data)

import pandas as pd
import numpy as np

class StrategyEngine:
    def __init__(self, data, exclude_star=True):
        """
        :param exclude_star: 是否剔除科创板 (代码 688 开头)，默认为 True
        """
        # 1. 填充空值
        self.data = data.fillna(0)
        
        # 2. 市场准入过滤 (Market Segment Veto)
        if exclude_star:
            # 确保 symbol 是字符串格式，并剔除 688 30 开头的标的
         
            initial_count = len(self.data)
            self.data = self.data[~self.data['symbol'].astype(str).str.startswith('688')]
            self.data = self.data[~self.data['symbol'].astype(str).str.startswith('30')]
            removed_count = initial_count - len(self.data)
            print(f"[Init] Market Filter: Excluded {removed_count} STAR Market (科创板) stocks.")

    def _normalize(self, series, ascending=True):
        """
        [Helper] 归一化函数：将数据映射到 0 - 100 分
        :param ascending: True 表示值越大分越高 (如 ROE)，False 表示值越小分越高 (如 PE)
        """
        # 1. Winsorization (去极值): 将超过 95% 和低于 5% 的异常值切掉
        # 这就像在物理实验中剔除 3-sigma 之外的噪声
        upper = series.quantile(0.95)
        lower = series.quantile(0.05)
        series_clipped = series.clip(lower, upper)

        # 2. Min-Max Scaling
        if ascending:
            score = (series_clipped - lower) / (upper - lower) * 100
        else:
            # 对于 PE 这种指标，越低越好，所以反向映射
            score = (upper - series_clipped) / (upper - lower) * 100
            
        return score.fillna(0) # 防止除以零产生的 NaN

    def apply_ranking(self, top_n=10):
        """
        策略 1: 均衡打分模型 (Balanced Scoring)
        使用了归一化算法，解决了指标量纲不一致的问题
        """
        print("[Analysis] Calculating Normalized Scores (Balanced Strategy)...")
        df = self.data.copy()

        # 1. 基础过滤 (Veto Cuts)
        # 剔除亏损 (PE<0)、剔除极端泡沫 (PE>80)、剔除微盘股 (市值<20亿)
        mask = (df['pe_ttm'] > 0) & (df['pe_ttm'] < 80) & (df['market_cap'] > 20_0000_0000)
        df = df[mask].copy()

        if df.empty: return pd.DataFrame()

        # 2. 计算各分项得分 (0-100分)
        # PE: 越低分越高
        df['score_val'] = self._normalize(df['pe_ttm'], ascending=False)
        # ROE: 越高分越高
        df['score_roe'] = self._normalize(df['roe'], ascending=True)
        # 增长率: 越高分越高
        df['score_growth'] = self._normalize(df['net_profit_growth'], ascending=True)
        # 现金流质量: 越高分越高 (新增)
        df['score_cash'] = self._normalize(df['cash_to_profit_ratio'], ascending=True)

        # 3. 加权总分 (Total Score)
        # 权重分配：盈利(40%) + 成长(30%) + 估值(20%) + 现金流(10%)
        df['total_score'] = (
            df['score_roe'] * 0.4 +
            df['score_growth'] * 0.3 +
            df['score_val'] * 0.2 +
            df['score_cash'] * 0.1
        )

        return df.sort_values(by='total_score', ascending=False).head(top_n)

    # =================================================================
    #  新增策略库
    # =================================================================

    def select_garp(self, top_n=10):
        """
        策略 2: GARP 策略 (Growth at a Reasonable Price)
        核心逻辑：寻找 PEG < 1 的股票 (增长速度 > 估值倍数)
        """
        print("[Analysis] Applying GARP Strategy (PEG < 1)...")
        df = self.data.copy()
        
        # 计算 PEG
        # 注意：避免除以 0 或负增长
        mask = (df['pe_ttm'] > 0) & (df['net_profit_growth'] > 0)
        df = df[mask].copy()
        
        df['peg'] = df['pe_ttm'] / df['net_profit_growth']
        
        # 筛选条件
        cuts = (
            (df['peg'] < 1.0) &               # 被低估
            (df['peg'] > 0.1) &               # 排除异常数据
            (df['roe'] > 12) &                # 必须有基本盈利能力
            (df['net_profit_growth'] > 15)    # 真正的成长股
        )
        
        result = df[cuts].sort_values(by='peg', ascending=True) # PEG 越小越好
        return result.head(top_n)

    def select_high_quality(self, top_n=10):
        """
        策略 3: “只买好公司” (High Quality / Buffett Style)
        核心逻辑：不看股价多贵，只看公司有多好 (高毛利、高ROE、低负债)
        """
        print("[Analysis] Applying High Quality Strategy (Moat)...")
        df = self.data.copy()
        
        cuts = (
            (df['roe'] > 20) &                # 极其优秀的盈利能力
            (df['gross_margin'] > 40) &       # 极高的毛利 (护城河)
            (df['debt_ratio'] < 50) &         # 低杠杆风险
            (df['cash_to_profit_ratio'] > 0.8)# 赚的是真钱
        )
        
        # 既然是好公司，按 ROE 排序，选最赚钱的
        result = df[cuts].sort_values(by='roe', ascending=False)
        return result.head(top_n)

    def select_deep_value(self, top_n=10):
        """
        策略 4: 深度价值 / 烟蒂股 (Deep Value)
        核心逻辑：价格跌无可跌，破净，但依然盈利
        """
        print("[Analysis] Applying Deep Value Strategy (Cigar Butt)...")
        df = self.data.copy()
        
        cuts = (
            (df['pe_ttm'] > 0) & (df['pe_ttm'] < 15) &  # 低 PE
            (df['pb'] > 0) & (df['pb'] < 1.5) &         # 低 PB (破净边缘)
            (df['roe'] > 8)                             # 还在赚钱，不是垃圾股
        )
        
        # 按 PB 排序，越便宜越好
        result = df[cuts].sort_values(by='pb', ascending=True)
        return result.head(top_n)


class RiskManager:
    def __init__(self, daq_engine):
        self.daq = daq_engine

    def assess_risk(self, symbol, current_price):
        """
        评估当前价格的风险等级
        """
        # 获取过去 90 天数据以计算 MA60
        df = self.daq.fetch_historical_kline(symbol, period='daily', start_date='20251001')
        
        if df.empty or len(df) < 60:
            return "Unknown (No Data)", 0.0

        # 计算均线
        ma20 = df['close'].rolling(window=20).mean().iloc[-1]
        ma60 = df['close'].rolling(window=60).mean().iloc[-1]
        
        # 1. 计算乖离率 (Bias Ratio)
        # 衡量当前价格偏离 60 日均线的程度
        bias_60 = (current_price - ma60) / ma60 * 100
        
        # 2. 判断状态
        if bias_60 > 30:
            status = "🔴 极度过热 (严重透支，做空/减仓区域)"
        elif bias_60 > 15:
            status = "🟠 偏高 (不要追高，等待回调)"
        elif 0 < bias_60 <= 15:
            status = "🟢 趋势健康 (持股/逢低吸纳)"
        elif -10 < bias_60 <= 0:
            status = "🔵 回调支撑位 (黄金买点)"
        else:
            status = "⚪ 超跌 (可能是基本面恶化)"
            
        return status, bias_60, ma20, ma60
# ==========================================
# 修正后的 Main Logic
# ==========================================
if __name__ == "__main__":
    db = StorageEngine()
    daq = DataEngine()
    
    # 获取今天日期
    today_str = datetime.now().strftime('%Y-%m-%d')
    print(f"[System] Current Date: {today_str}")

    # ---------------------------------------------------------
    # Step 1: 市场行情缓存检查
    # ---------------------------------------------------------
    latest_market_date = db.get_latest_date('market_snapshot', 'fetch_date')
    print(f"[Debug] DB Market Date: {latest_market_date}")

    if latest_market_date == today_str:
        print(f"[Cache] Today's market snapshot found. Loading...")
        market_data = db.load_from_sql('market_snapshot')
    else:
        # 只有日期不匹配时才会打印这两行
        print(f"[DAQ] Today's snapshot not found. Fetching from API...")
        market_data = daq.fetch_market_snapshot()
        market_data['fetch_date'] = today_str # 注入日期标签
        db.save_to_sql(market_data, 'market_snapshot')

    # ---------------------------------------------------------
    # Step 2: 深度财务数据缓存检查 (这是最耗时的部分)
    # ---------------------------------------------------------
    # 检查【全量分析表】今天是否已经生成过
    latest_full_date = db.get_latest_date('full_analysis_table', 'fetch_date')
    print(f"[Debug] DB Full Analysis Date: {latest_full_date}")

    if latest_full_date == today_str:
        print(f"[Cache] Today's deep analysis found. Loading directly...")
        full_data = db.load_from_sql('full_analysis_table')
    else:
        # 只有在没有缓存时，才执行耗时的 API 循环抓取
        print(f"[Trigger] No cache for today. Starting Level 1 Trigger...")
        
        # 粗筛候选单
        candidates = market_data[
            (market_data['pe_ttm'] < 50) & 
            (market_data['market_cap'] > 50_0000_0000)
        ]
        # target_symbols = candidates['symbol'].tolist()[:2] 
        target_symbols = candidates['symbol'].tolist()
        # 执行昂贵的抓取操作
        finance_data = daq.fetch_detailed_financials(target_symbols)
        
        if not finance_data.empty:
            full_data = pd.merge(candidates, finance_data, on='symbol', how='inner')
            full_data['fetch_date'] = today_str # 注入日期标签
            db.save_to_sql(full_data, 'full_analysis_table')
            print(f"[Storage] Deep analysis table cached for {today_str}")
        else:
            full_data = pd.DataFrame()
            print("[Error] Failed to fetch financial data.")

    # ---------------------------------------------------------
    # Step 3: 执行最终分析
    # ---------------------------------------------------------
    # ... (前面的 Step 1, 2, 3 不变) ...
    
    if not full_data.empty:
        # 打印一下诊断信息
        # print("\n[Diagnostic] 原始数据概览:")
        # print(full_data[['symbol', 'pe_ttm', 'roe', 'net_profit_growth']].describe())

        analyzer = StrategyEngine(full_data)

        # 1. 运行均衡归一化打分
        res_score = analyzer.apply_ranking(top_n=50)
        print("\n" + "="*60)
        print(f"📊 策略 A: 均衡打分 Top 10 (Normalized)")
        print("="*60)
        cols_score = ['symbol', 'name', 'price', 'total_score', 'pe_ttm', 'roe', 'net_profit_growth']
        print(res_score[cols_score].to_string(index=False))

        # 2. 运行 GARP 策略
        res_garp = analyzer.select_garp(top_n=5)
        print("\n" + "-"*60)
        print(f"🚀 策略 B: GARP 成长 (PEG < 1)")
        print("-" * 60)
        if not res_garp.empty:
            print(res_garp[['symbol', 'name','price', 'peg', 'pe_ttm', 'net_profit_growth']].to_string(index=False))
        else:
            print("无符合 GARP 标准的标的")

        # 3. 运行高质量护城河策略
        res_quality = analyzer.select_high_quality(top_n=5)
        print("\n" + "-"*60)
        print(f"🏰 策略 C: 护城河 (高 ROE + 高毛利)")
        print("-" * 60)
        if not res_quality.empty:
            print(res_quality[['symbol', 'name','price', 'roe', 'gross_margin', 'debt_ratio']].to_string(index=False))
        else:
            print("无符合护城河标准的标的")

    db.close()




[System] Current Date: 2026-01-11
[Debug] DB Market Date: 2026-01-11
[Cache] Today's market snapshot found. Loading...
[Debug] DB Full Analysis Date: 2026-01-11
[Cache] Today's deep analysis found. Loading directly...
[Init] Market Filter: Excluded 359 STAR Market (科创板) stocks.
[Analysis] Calculating Normalized Scores (Balanced Strategy)...


🛑 风险评估报告 (基于均值回归理论)
代码       名称       现价       乖离率(%)     MA60成本     操作建议
--------------------------------------------------------------------------------


AttributeError: 'DataEngine' object has no attribute 'fetch_historical_kline'

In [3]:
import akshare as ak
import pandas as pd
import sqlite3
from datetime import datetime
from tqdm import tqdm
import os
import time
import warnings

# 忽略 pandas 的 FutureWarning
warnings.filterwarnings('ignore')

# ==========================================
# 模块 1: StorageEngine (数据库管理)
# ==========================================
class StorageEngine:
    def __init__(self, db_path='data/stock_market.db'):
        os.makedirs(os.path.dirname(db_path), exist_ok=True)
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()

    def save_to_sql(self, df, table_name, if_exists='replace'):
        try:
            df.to_sql(table_name, self.conn, if_exists=if_exists, index=False)
            print(f"[Storage] Table '{table_name}' saved. Rows: {len(df)}")
        except Exception as e:
            print(f"[Storage] Error saving {table_name}: {e}")

    def load_from_sql(self, table_name):
        try:
            return pd.read_sql(f"SELECT * FROM {table_name}", self.conn)
        except Exception as e:
            return pd.DataFrame()
    
    def table_exists(self, table_name):
        self.cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
        return self.cursor.fetchone() is not None
    
    def get_latest_date(self, table_name, date_column='fetch_date'):
        if not self.table_exists(table_name): return None
        try:
            self.cursor.execute(f"SELECT MAX({date_column}) FROM {table_name}")
            return self.cursor.fetchone()[0]
        except: return None
    
    def close(self):
        self.conn.close()

# ==========================================
# 模块 2: DataEngine (数据采集 - 终极修正版)
# ==========================================
class DataEngine:
    def __init__(self):
        self.request_interval = 0.5 

    def fetch_market_snapshot(self):
        print("[DAQ] Fetching Real-time Market Snapshot...")
        try:
            df_spot = ak.stock_zh_a_spot_em()
            rename_map = {
                '代码': 'symbol', '名称': 'name', 
                '最新价': 'price', '涨跌幅': 'change_pct',
                '市盈率-动态': 'pe_ttm', '市净率': 'pb', 
                '总市值': 'market_cap', '换手率': 'turnover'
            }
            df_spot = df_spot.rename(columns=rename_map)
            numeric_cols = ['pe_ttm', 'pb', 'market_cap', 'change_pct', 'price']
            for col in numeric_cols:
                if col in df_spot.columns:
                    df_spot[col] = pd.to_numeric(df_spot[col], errors='coerce')
            return df_spot
        except Exception as e:
            print(f"[Error] Market snapshot failed: {e}")
            return pd.DataFrame()

    def fetch_historical_kline(self, symbol, period='daily', start_date='20251001'):
        """获取K线数据用于计算乖离率"""
        try:
            # 适配 akshare 接口变化，尝试自动补全前缀
            df = ak.stock_zh_a_hist(symbol=symbol, period=period, start_date=start_date, adjust="qfq")
            if df.empty: return pd.DataFrame()
            
            df = df.rename(columns={'日期':'date', '开盘':'open', '收盘':'close', 
                                  '最高':'high', '最低':'low', '成交量':'volume'})
            return df
        except:
            return pd.DataFrame()

    def fetch_detailed_financials(self, symbol_list):
        print(f"[DAQ] Starting Deep Reconstruction for {len(symbol_list)} stocks...")
        financial_data = []
        
        for symbol in tqdm(symbol_list):
            try:
                df = None
                source = 'em'
                
                # 1. 尝试 EM 接口
                try:
                    df = ak.stock_financial_analysis_indicator(symbol=symbol)
                    if df is None or df.empty: raise ValueError
                except:
                    # 2. 尝试 THS 接口
                    try:
                        source = 'ths'
                        df = ak.stock_financial_abstract_ths(symbol=symbol, indicator="按报告期")
                    except:
                        continue 

                if df is None or df.empty: continue

                # --- 关键修正 1: 强制按日期降序排列 (防止取到2011年数据) ---
                date_col = None
                for col in df.columns:
                    if '日期' in str(col) or '报告期' in str(col):
                        date_col = col
                        break
                
                if date_col:
                    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
                    df = df.sort_values(by=date_col, ascending=False)
                
                latest = df.iloc[0] # 获取最新一行

                # --- 关键修正 2: 模糊匹配 (防止列名不匹配返回0) ---
                def get_fuzzy(keywords_list):
                    for col_name in latest.index:
                        if any(k in str(col_name) for k in keywords_list):
                            raw_val = latest[col_name]
                            if isinstance(raw_val, bool): return 0.0 # 过滤布尔值
                            val_str = str(raw_val).replace('%', '').replace(',', '').replace('万', '').replace('亿', '')
                            try:
                                return float(val_str)
                            except:
                                return 0.0
                    return 0.0

                data_point = {'symbol': symbol}
                data_point['roe'] = get_fuzzy(['净资产收益率', 'ROE'])
                data_point['gross_margin'] = get_fuzzy(['毛利率', '销售毛利'])
                data_point['net_profit_growth'] = get_fuzzy(['净利润同比增长', '净利润增长', '归属净利润同比'])
                data_point['debt_ratio'] = get_fuzzy(['资产负债率', '负债合计'])
                data_point['eps'] = get_fuzzy(['基本每股收益', '每股收益'])
                data_point['ocf_per_share'] = get_fuzzy(['每股经营活动', '每股经营现金', '经营净现金流/股'])
                
                eps = data_point['eps']
                data_point['cash_to_profit_ratio'] = (data_point['ocf_per_share'] / eps) if eps != 0 else 0
                
                financial_data.append(data_point)
                time.sleep(0.3 if source == 'em' else 0.1)
                
            except Exception as e:
                continue
                    
        return pd.DataFrame(financial_data)

# ==========================================
# 模块 3: StrategyEngine (选股策略)
# ==========================================
class StrategyEngine:
    def __init__(self, data, exclude_star=True):
        """
        :param exclude_star: True 则自动剔除科创板(688)和创业板(300)
        """
        self.data = data.fillna(0)
        
        if exclude_star:
            # 市场准入过滤
            initial_count = len(self.data)
            # 剔除 688 和 300 开头的股票
            self.data = self.data[
                ~self.data['symbol'].astype(str).str.startswith(('688', '30', '92', '8', '4'))
            ]
            removed_count = initial_count - len(self.data)
            print(f"[Init] Market Filter: Excluded {removed_count} STAR/ChiNext stocks.")

    def _normalize(self, series, ascending=True):
        if series.empty: return series
        upper = series.quantile(0.95)
        lower = series.quantile(0.05)
        if upper == lower: return series * 0
        series_clipped = series.clip(lower, upper)

        if ascending:
            score = (series_clipped - lower) / (upper - lower) * 100
        else:
            score = (upper - series_clipped) / (upper - lower) * 100
        return score.fillna(0)

    def apply_ranking(self, top_n=20):
        """策略 A: 均衡打分"""
        print("[Analysis] Calculating Balanced Scores...")
        df = self.data.copy()
        
        # 基础过滤
        mask = (df['pe_ttm'] > 0) & (df['pe_ttm'] < 80) & (df['market_cap'] > 20_0000_0000)
        df = df[mask].copy()
        if df.empty: return pd.DataFrame()

        # 归一化
        df['score_val'] = self._normalize(df['pe_ttm'], ascending=False)
        df['score_roe'] = self._normalize(df['roe'], ascending=True)
        df['score_growth'] = self._normalize(df['net_profit_growth'], ascending=True)
        df['score_cash'] = self._normalize(df['cash_to_profit_ratio'], ascending=True)

        # 综合打分
        df['total_score'] = (
            df['score_roe'] * 0.4 +
            df['score_growth'] * 0.3 +
            df['score_val'] * 0.2 +
            df['score_cash'] * 0.1
        )
        if top_n > 0:
            return df.sort_values(by='total_score', ascending=False).head(top_n)
        return df.sort_values(by='total_score', ascending=False)

    def select_garp(self, top_n=10):
        """策略 B: GARP 成长"""
        print("[Analysis] Applying GARP Strategy...")
        df = self.data.copy()
        mask = (df['pe_ttm'] > 0) & (df['net_profit_growth'] > 0)
        df = df[mask].copy()
        df['peg'] = df['pe_ttm'] / df['net_profit_growth']
        
        cuts = (
            (df['peg'] < 1.0) & (df['peg'] > 0.1) & 
            (df['roe'] > 12) & (df['net_profit_growth'] > 15)
        )
        if top_n > 0:
            return df[cuts].sort_values(by='peg', ascending=True).head(top_n)
        return df[cuts].sort_values(by='peg', ascending=True)
    
    def select_high_quality(self, top_n=10):
        """策略 C: 护城河"""
        print("[Analysis] Applying High Quality Strategy...")
        df = self.data.copy()
        cuts = (
            (df['roe'] > 15) & (df['gross_margin'] > 30) & 
            (df['debt_ratio'] < 60) & (df['cash_to_profit_ratio'] > 0.8)
        )
        if top_n > 0:
            return df[cuts].sort_values(by='roe', ascending=False).head(top_n)
        return df[cuts].sort_values(by='roe', ascending=False)
    

# ==========================================
# 模块 4: RiskManager (风控与择时)
# ==========================================
class RiskManager:
    def __init__(self, daq_engine: DataEngine):
        self.daq = daq_engine

    def assess_risk(self, symbol, current_price):
        """
        计算乖离率，判断是否过热
        """
        # 获取 K 线 (过去 4 个月)
        df = self.daq.fetch_historical_kline(symbol, period='daily', start_date='20250601')
        
        if df.empty or len(df) < 60:
            return "⚪ 数据不足", 0.0, 0.0

        # 计算均线
        ma20 = df['close'].rolling(window=20).mean().iloc[-1]
        ma60 = df['close'].rolling(window=60).mean().iloc[-1]
        
        # 计算乖离率 Bias (现价偏离季线的程度)
        bias_60 = (current_price - ma60) / ma60 * 100
        
        # 判定红绿灯
        if bias_60 > 30:
            status = "🔴 极度过热 (严重透支，建议回避)"
        elif bias_60 > 10:
            status = "🟠 偏高 (不要追高，等待回调)"
        elif 0 < bias_60 <= 10:
            status = "🟢 趋势健康 (持股/逢低吸纳)"
        elif -10 < bias_60 <= 0:
            status = "🔵 回调支撑位 (黄金买点)"
        else:
            status = "⚪ 超跌/弱势 (需结合基本面)"
            
        return status, bias_60, ma60
class MoneyFlowAnalyzer:
    def __init__(self):
        pass

    def get_money_flow(self, symbol):
        """
        [维度1] 获取个股资金流向 (计算大单净流入占比)
        """
        try:
            # akshare 接口：个股资金流向
            # 注意：这个接口返回的是历史数据，我们需要取最近的一天
            df = ak.stock_individual_fund_flow(stock=symbol, market="sh" if symbol.startswith('6') else "sz")
            if df.empty: return 0.0, "无数据"
            
            # 这里的列名通常包括：日期, 主力净流入-净额, 主力净流入-净占比...
            # 我们按日期降序
            df = df.sort_values(by='日期', ascending=False)
            latest = df.iloc[0]
            
            # 获取主力净占比 (Net Flow Ratio)
            # 如果 > 0 说明主力在买，> 10% 说明主力在大买
            net_ratio = float(latest['主力净流入-净占比'])
            
            if net_ratio > 10:
                status = "🔥 主力抢筹"
            elif net_ratio > 0:
                status = "🔴 主力小买"
            elif net_ratio > -10:
                status = "🟢 主力小卖"
            else:
                status = "🧊 主力出逃"
                
            return net_ratio, status
        except:
            return 0.0, "数据缺失"

    def check_dragon_tiger(self, symbol, date_str):
        """
        [维度2] 检查最近是否上过龙虎榜 (LHB)
        """
        try:
            # 获取指定日期的龙虎榜详情
            # 这是一个比较重的查询，实际策略中建议只查 Top 股
            lhb_df = ak.stock_lhb_detail_em(start_date=date_str, end_date=date_str)
            
            if symbol in lhb_df['代码'].values:
                # 如果上榜，进一步分析是谁在买
                target_row = lhb_df[lhb_df['代码'] == symbol].iloc[0]
                
                buy_amt = float(target_row['买入总计'])
                sell_amt = float(target_row['卖出总计'])
                net_buy = buy_amt - sell_amt
                
                return True, net_buy
            return False, 0.0
        except:
            return False, 0.0
# ==========================================
# 辅助函数: 打印漂亮的报表
# ==========================================

def print_report(df: pd.DataFrame, risk_mgr: RiskManager, flow_analyzer: MoneyFlowAnalyzer, title: str):
    if df.empty:
        print(f"\n--- {title} 无匹配标的 ---")
        return

    # 1. 数据增强：先计算所有指标，存入临时 DataFrame 以便排序
    report_list = []
    
    print(f"\n[Report] Analyzing {len(df)} candidates for {title}...")
    
    for _, row in df.iterrows():
        symbol = row['symbol']
        price = row.get('price', 0)
        
        # 技术面
        status_tech, bias, ma60 = risk_mgr.assess_risk(symbol, price)
        # 资金面
        mf_ratio, mf_status = flow_analyzer.get_money_flow(symbol)
        
        # --- 核心决策逻辑 (Decision Matrix) ---
        # 评分系统：资金流权重高，位置权重次之
        
        signal = "⚪ 观望"
        rank_val = 0 # 用于排序
        
        if mf_ratio > 5: # 主力在大买
            if bias < 0:
                signal = "💎 底部共振 (极品)"
                rank_val = 100
            elif 0 <= bias < 15:
                signal = "🚀 趋势加速 (进攻)"
                rank_val = 90
            else:
                signal = "⚠ 资金诱多 (过热)"
                rank_val = 40
        elif mf_ratio > 0: # 主力小买
            if bias < 0:
                signal = "💰 低吸潜伏"
                rank_val = 80
            elif 0 <= bias < 15:
                signal = "✅ 顺势持股"
                rank_val = 70
            else:
                signal = "🟠 分批止盈"
                rank_val = 50
        else: # 主力在卖
            if bias > 20:
                signal = "☠ 顶部背离 (快跑)"
                rank_val = -10
            else:
                signal = "🧊 阴跌风险"
                rank_val = 10
                
        report_list.append({
            'symbol': symbol,
            'name': row['name'],
            'price': price,
            'pe': row['pe_ttm'],
            'roe': row['roe'],
            'bias': bias,
            'ma60': ma60,
            'mf_ratio': mf_ratio,
            'signal': signal,
            'rank': rank_val
        })
        
    # 2. 排序：按 rank (推荐度) 降序，其次按 资金流 降序
    report_df = pd.DataFrame(report_list)
    report_df = report_df.sort_values(by=['rank', 'mf_ratio'], ascending=[False, False])

    # 3. 打印漂亮报表
    print("\n" + "="*115)
    print(f"📊 {title} (已按推荐度排序)")
    print("="*115)
    
    # 动态表头
    headers = f"{'代码':<8} {'名称':<8} {'现价':<8} {'PE':<8} {'ROE':<6} {'乖离率%':<8} {'MA60':<8} {'主力净流%':<10} {'AI决策建议'}"
    print(headers)
    print("-" * 115)

    for _, row in report_df.iterrows():
        # 颜色标记 (仅在支持ANSI的终端有效，普通终端会显示为普通文本)
        # 稍微调整一下格式，让它更像专业软件
        print(f"{row['symbol']:<8} {row['name']:<8} {row['price']:<8.2f} {row['pe']:<8.1f} {row['roe']:<6.1f} {row['bias']:<8.1f} {row['ma60']:<8.2f} {row['mf_ratio']:<10.2f} {row['signal']}")
    
    print("="*115)

# ==========================================
# Main Logic
# ==========================================
if __name__ == "__main__":
    db = StorageEngine()
    daq = DataEngine()
    
    today_str = datetime.now().strftime('%Y-%m-%d')
    print(f"[System] Date: {today_str}")

    # 1. 市场行情 (Level 1)
    latest_market_date = db.get_latest_date('market_snapshot', 'fetch_date')
    if latest_market_date == today_str:
        print(f"[Cache] Loading market snapshot...")
        market_data = db.load_from_sql('market_snapshot')
    else:
        market_data = daq.fetch_market_snapshot()
        market_data['fetch_date'] = today_str
        db.save_to_sql(market_data, 'market_snapshot')

    # 2. 财务数据 (Level 3)
    latest_full_date = db.get_latest_date('full_analysis_table', 'fetch_date')
    
    if latest_full_date == today_str:
        print(f"[Cache] Loading deep analysis data...")
        full_data = db.load_from_sql('full_analysis_table')
    else:
        print(f"[Trigger] Pre-selecting candidates...")
        # 粗筛：PE < 60 且市值 > 30亿 (稍微放宽一点，避免筛空)
        candidates = market_data[
            (market_data['pe_ttm'] < 60) & 
            (market_data['market_cap'] > 30_0000_0000)
        ]
    
        target_symbols = candidates['symbol'].tolist()
        
        finance_data = daq.fetch_detailed_financials(target_symbols)
        
        if not finance_data.empty:
            full_data = pd.merge(candidates, finance_data, on='symbol', how='inner')
            full_data['fetch_date'] = today_str
            db.save_to_sql(full_data, 'full_analysis_table')
            print(f"[Storage] Saved {len(full_data)} rows.")
        else:
            full_data = pd.DataFrame()
            print("[Error] Financial data fetch failed.")

    # 3. 策略分析与风控 (Analysis & Risk)
    if not full_data.empty:
        # 初始化策略引擎 (exclude_star=True 剔除科创/创业板)
        analyzer = StrategyEngine(full_data, exclude_star=True)
        # 初始化风控引擎
        risk_mgr = RiskManager(daq)
        flow_mgr = MoneyFlowAnalyzer()
        # 运行策略 A: 均衡打分
        res_score = analyzer.apply_ranking(top_n=0)
        print_report(res_score, risk_mgr,flow_mgr, "策略 A: 均衡优选 (综合评分)")

        # 运行策略 B: GARP 成长
        res_garp = analyzer.select_garp(top_n=200)
        print_report(res_garp, risk_mgr,flow_mgr, "策略 B: GARP 成长 (低PEG)")

        # 运行策略 C: 护城河
        res_quality = analyzer.select_high_quality(top_n=200)
        print_report(res_quality, risk_mgr,flow_mgr, "策略 C: 优质护城河 (高ROE)")

    db.close()


[System] Date: 2026-01-13
[DAQ] Fetching Real-time Market Snapshot...


  0%|          | 0/57 [00:00<?, ?it/s]

[Storage] Table 'market_snapshot' saved. Rows: 5797
[Trigger] Pre-selecting candidates...
[DAQ] Starting Deep Reconstruction for 3200 stocks...


  3%|▎         | 93/3200 [00:52<31:09,  1.66it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

100%|██████████| 3200/3200 [19:45<00:00,  2.70it/s] 


[Storage] Table 'full_analysis_table' saved. Rows: 2228
[Storage] Saved 2228 rows.
[Init] Market Filter: Excluded 852 STAR/ChiNext stocks.
[Analysis] Calculating Balanced Scores...

[Report] Analyzing 986 candidates for 策略 A: 均衡优选 (综合评分)...

📊 策略 A: 均衡优选 (综合评分) (已按推荐度排序)
代码       名称       现价       PE       ROE    乖离率%     MA60     主力净流%      AI决策建议
-------------------------------------------------------------------------------------------------------------------
603600   永艺股份     11.63    15.1     8.4    -2.6     11.94    15.29      💎 底部共振 (极品)
605028   世茂能源     0.00     35.8     8.6    -100.0   28.45    14.70      💎 底部共振 (极品)
001359   平安电工     71.32    49.4     10.6   -0.3     71.53    12.98      💎 底部共振 (极品)
600803   新奥股份     20.09    13.6     14.0   -4.4     21.01    12.48      💎 底部共振 (极品)
601577   长沙银行     9.50     4.4      9.4    -1.6     9.65     11.97      💎 底部共振 (极品)
603312   西典新能     47.56    27.1     12.0   -0.4     47.77    10.37      💎 底部共振 (极品)
002984   森麒麟      21.01    16